# Final Exam <font color='red'>PART B</font> | ITSS 4382 Applied AI/ Machine Learning

## <font color='red'>Open Notes, Open Internet</font>
## <font color='red'>However, the use of ChatGPT or any external language model for directly completing or answering questions is strictly prohibited and will result in a grade of </font>ZERO<font color='red'>. Students should understand the consequences of violating this policy and the expectation of independent work during the test. Well done!</font><br><br>



<font color='#ccc'>**PART A:** Answer the MCQ Q1-Q20.</font><br>
<font color='red'>**PART B:** Answer the following questions and write your own code/answers in the boxes below.</font><br>

In this part of the exam, you need to 
 - write down your codes, printed results, and corresponding discussions in the cells below the questions.
 - add extra cells if needed using the "+" button or shortcut "esc + b", "alt + enter". You do NOT need to remove these cells upon submission.
 - submit your compiled .ipynb file to eLearning.

In [1]:
YOUR_NAME = "Winston Shih"

In [2]:
%matplotlib inline 
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# <font color='red'>PART B1. Gender Recognition by Speech Analysis</font> 
You need to include both codes and discussions in this coding file.


##### Setting and Data
This dataset is created to identify a voice as male or female, based upon acoustic properties of the voice and speech. It consists of 3,168 recorded voice samples, collected from male and female speakers. The voice samples are pre-processed by acoustic analysis, with an analyzed frequency range of 0Hz-280Hz (human vocal range).

The CSV file contains 20 acoustic properties of each voice, and one outcome variable, “label”, which identifies the gender of the speaker. The detailed information is listed below (you do NOT need to read through the variable description). 

- meanfreq: mean frequency (in kHz)
- sd: standard deviation of frequency
- median: median frequency (in kHz)
- Q25: first quantile (in kHz)
- Q75: third quantile (in kHz)
- IQR: interquantile range (in kHz)
- skew: skewness (see note in specprop description)
- kurt: kurtosis (see note in specprop description)
- sp.ent: spectral entropy
- sfm: spectral flatness
- mode: mode frequency
- centroid: frequency centroid (see specprop)
- meanfun: average of fundamental frequency measured across acoustic signal
- minfun: minimum fundamental frequency measured across acoustic signal
- maxfun: maximum fundamental frequency measured across acoustic signal
- meandom: average of dominant frequency measured across acoustic signal
- mindom: minimum of dominant frequency measured across acoustic signal
- maxdom: maximum of dominant frequency measured across acoustic signal
- dfrange: range of dominant frequency measured across acoustic signal
- modindx: modulation index. Calculated as the accumulated absolute difference between adjacent measurements of fundamental frequencies divided by the frequency range
- label: male or female

## Preliminaries
Use the code below to load data and check the variable names.


In [3]:
import pandas as pd
voice = pd.read_csv('voice.csv') 
voice['label'] = voice['label'].astype('category').cat.codes
voice.describe()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
count,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,...,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000
mean,0.180907,0.057126,0.185621,0.140456,0.224765,0.084309,3.140168,36.568461,0.895127,0.408216,...,0.180907,0.142807,0.036802,0.258842,0.829211,0.052647,5.047277,4.994630,0.173752,0.500000
std,0.029918,0.016652,0.036360,0.048680,0.023639,0.042783,4.240529,134.928661,0.044980,0.177521,...,0.029918,0.032304,0.019220,0.030077,0.525205,0.063299,3.521157,3.520039,0.119454,0.500079
min,0.039363,0.018363,0.010975,0.000229,0.042946,0.014558,0.141735,2.068455,0.738651,0.036876,...,0.039363,0.055565,0.009775,0.103093,0.007812,0.004883,0.007812,0.000000,0.000000,0.000000
25%,0.163662,0.041954,0.169593,0.111087,0.208747,0.042560,1.649569,5.669547,0.861811,0.258041,...,0.163662,0.116998,0.018223,0.253968,0.419828,0.007812,2.070312,2.044922,0.099766,0.000000
50%,0.184838,0.059155,0.190032,0.140286,0.225684,0.094280,2.197101,8.318463,0.901767,0.396335,...,0.184838,0.140519,0.046110,0.271186,0.765795,0.023438,4.992188,4.945312,0.139357,0.500000
75%,0.199146,0.067020,0.210618,0.175939,0.243660,0.114175,2.931694,13.648905,0.928713,0.533676,...,0.199146,0.169581,0.047904,0.277457,1.177166,0.070312,7.007812,6.992188,0.209183,1.000000
max,0.251124,0.115273,0.261224,0.247347,0.273469,0.252225,34.725453,1309.612887,0.981997,0.842936,...,0.251124,0.237636,0.204082,0.279114,2.957682,0.458984,21.867188,21.843750,0.932374,1.000000


We would like to use all other variables to predict the gender of the speaker (label). To start, we split the data using the  following code.

In [4]:
from sklearn.model_selection import train_test_split

X = voice.iloc[:, 0:19]
y = voice.iloc[:, 20]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## Q1. Scaling (5 Pts)

Many machine learning methods require data scaling. In this problem, we scale our data first. 
- Use standard scaler to scale the data properly, so that the data can be applied for **supervised learning models**.
- Note: Use the scaled data for all classification models (i.e., Q2 – Q5).

In [5]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)






In [6]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape






((2376, 19), (2376,), (792, 19), (792,))

## Q2. Simple Models - Naïve Bayes and Decision Tree (8 pts)

- Train a Decision Tree classifier. Let max_depth be 2, and min_samples_split be 10. Set random_state to 0. What is the training accuracy? What is the test accuracy? (4 pts)

- Train a Gaussian Naïve Bayes classifier. Leave all specifications as default. What is the training accuracy? What is the test accuracy? (4 pts)



In [7]:
from sklearn.tree import DecisionTreeClassifier

dt=DecisionTreeClassifier(random_state=0, max_depth=2, min_samples_split=10)
dt.fit(X_train, y_train)
dt.score(X_test, y_test), dt.score(X_train, y_train)








(0.9532828282828283, 0.9553872053872053)

In [8]:
#Training accuracy is 0.9553872053872053. Test accuracy is 0.9532828282828283. 

In [9]:
from sklearn.naive_bayes import GaussianNB
g_nb=GaussianNB()
g_nb.fit(X_train, y_train)

GaussianNB()

In [10]:
g_nb.score(X_test, y_test), g_nb.score(X_train, y_train)

(0.8876262626262627, 0.8901515151515151)

In [11]:
#Training accuracy is 0.8901515151515151. Testing accuracy is 0.8876262626262627.


## Q3. Logistic Regression (10 pts)

- Train a logistic regression classifier and L2 regularization. Let random state be 0 and alpha be 1. What is the training accuracy? What is the test accuracy? (4 pts)
- Report coefficient and confusion_matrix of the above model. (4 pts)
- Suppose we want to train another logistic regression classifier with alpha = 100. Without actually training the new model, do you expect its training accuracy to be higher or lower than the previous logistic regression model (where alpha = 1)? Explain briefly.(2 pts)



In [12]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(penalty='l2', random_state=0, C=1)
logreg.fit(X_train_scaled, y_train)
y_pred = logreg.predict(X_test_scaled)
print(f"Training Set Accuracy: {logreg.score(X_train_scaled, y_train):.2f}")
print(f"Test Set Accuracy: {logreg.score(X_test_scaled, y_test):.2f}")

Training Set Accuracy: 0.97
Test Set Accuracy: 0.98


In [13]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print("coef is: ", logreg.coef_)








coef is:  [[-6.45155911e-01  1.51757724e+00 -8.01840781e-01 -5.72706493e+00
   2.73454822e+00  8.46161316e+00 -3.69420786e-01  1.06647414e-02
   4.04723967e+00 -1.07145506e+00  1.04883545e+00 -6.45155911e-01
  -1.41493862e+01 -5.37550667e-02 -2.42743999e+00 -2.07347799e-01
  -1.56904830e+00 -7.94611496e-01  7.74436803e-01]]


In [14]:
from sklearn.metrics import confusion_matrix
my_matrix = confusion_matrix(y_test, y_pred)

print("TP is:", my_matrix[1,1])
print("TN is:", my_matrix[0,0])
print("FP is:", my_matrix[0,1])
print("FN is:", my_matrix[1,0])









TP is: 414
TN is: 359
FP is: 14
FN is: 5


In [15]:
#Training accuracy would be lower if alpha increased to 100 because if alpha is increased too high, then the model 
#could underfit the data.

## Q4. Ensemble Method I (10 pts) 

- Train a Random Forest model. Let n_estimators be 200, max_depth be 2, and min_samples_split be 10. Set random_state to 0. What is the training accuracy? What is the test accuracy? (4 pts)
- Train an Adaboost classifier, where each single model is a decision tree with max_depth = 2, min_samples_split = 10, and random state = 0. let n_estimators be 200. What is the training accuracy? What is the test accuracy? (4 pts)
- Briefly discuss the similarities and differences between the two models we trained (i.e., Random Forest and Adaboost with decision tree as base model). (2 pts)

In [16]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf=RandomForestClassifier(n_estimators=200, min_samples_split=10, 
                               random_state=0)
rnd_clf.fit(X_train, y_train)
print('Training accuracy is: ')
print(rnd_clf.score(X_train, y_train))








Training accuracy is: 
0.9945286195286195


In [17]:
print('Testing accuracy is: ')
print(rnd_clf.score(X_test, y_test))








Testing accuracy is: 
0.9797979797979798


In [18]:
from sklearn.ensemble import AdaBoostClassifier
naive_dt=DecisionTreeClassifier(max_depth=2, min_samples_split=10)
ada_clf = AdaBoostClassifier(
    naive_dt, n_estimators=200, 
    random_state=0)
ada_clf.fit(X_train, y_train)
print('Train accuracy is: ')
print(ada_clf.score(X_train, y_train))





Train accuracy is: 
1.0


In [19]:
print('Test accuracy is: ')
print(ada_clf.score(X_test, y_test))

Test accuracy is: 
0.9848484848484849


In [20]:
#The similarities between RandomForest and Ada Classifier are that they are Ensemble Algorithms that are used for Classifying
#Data. However, RandomForest is a Bagging Classifier that is used to reduce variance of weaker learners. AdaBoost is a 
#BoostStrap Classifier that is supposed reduce bias of weak learners. RandomForest uses all DecisionTree outputs to produce a  
#combined output that can reduce overfitting, encourage decision tree diversity, and improve accuracy. AdaBoost increases 
# accuracy by using every n_estimator trial to redistribute weight of trees, so incorrectly classified conditions are given more weight 
#than the conditions that are correctly classified.

## Q5. Ensemble Method II (8 pts)

For now, let us focus on three simple models (without ensembling) we have trained: Decision Tree, Naïve Bayes, and Logistic Regression.
- Use voting classifier (with hard voting) that includes the above-mentioned three models. (4 pts)
- What is the training and test accuracy of the voting classifier? Is the test score of the voting classifier better than other models? Briefly explain why the voting classifier is/is not doing better. (3 pts)

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
log_clf = LogisticRegression(random_state=0)
dt_clf = DecisionTreeClassifier(random_state=0, criterion='entropy')
nbg_clf = GaussianNB()
from sklearn.ensemble import VotingClassifier
# define voting classifier
voting_clf = VotingClassifier(estimators = [('lr1', log_clf), ('dt', dt_clf), ('nbg', nbg_clf)], voting ='hard')
voting_clf.fit(X_train, y_train)
print("Training score for voting classifier is:", voting_clf.score(X_train, y_train))
print("Test score for voting classifier is:", voting_clf.score(X_test, y_test))










Training score for voting classifier is: 0.9448653198653199
Test score for voting classifier is: 0.9444444444444444


In [22]:
#The training and testing score of voting classifier is worse than the the scores of AdaBoost,RandomForest, Logistic Regression,
#and Decision Tree Classifier. However, its testing and training scores are better than Naive Bayes. The voting classifier is 
# niot doing better because it includes the Naive Bayes model, which is less accurate than the other  models and caused accuracy
#of overall classifier to be lower.





# <font color='red'>PART B2: Clustering (10 pts)</font>

Consider another scenario where we only collect the acoustic information and fail to collect the gender information (e.g., in an online setting, some users prefer not to disclose their gender). This case, we may consider using clustering methods to find groups of acoustic information similar to each other. 
- Apply MinMax Scaling to our data, so that the data can be applied for **unsupervised learning models.** (4 pts)
- Hint: We do not need to split the data for unsupervised learning. Thus, the scaler should be trained on the full sample (i.e., X) rather than the training sample (i.e., X_train).
- Train a K-Means model, set k=3, and random state = 0. (4 pts）
- How many clusters do we have? Why? (2 pts)

In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler1=MinMaxScaler()
X_scaled = scaler.fit_transform(X)




In [24]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(X_scaled)
cls_predict = kmeans.predict(X_scaled)





In [25]:
print("Number of clusters:", kmeans.n_clusters)

Number of clusters: 3


In [26]:
#There are 3 clusters because it is the optimal number of clusters where the K Means' testing and training accuracy is 
#most accurate.